# Importing initial libraries


In [ ]:

import datetime
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

# Downloading and saving data


In [ ]:
# Set up yfinance to use pandas data reader
yf.pdr_override()

# Define the list of tickers
tickers = ['COR', 'CE', 'HUM', 'CINF', 'KEYS', 'NXPI', 'BMY', 'EMR',
           'CVS', 'RVTY', 'ES', 'DHI', 'ZBRA', 'KMI', 'INVH', 'GPC', 'SWKS', 'GIS']

# Define the date range
start_date = '2002-01-01'
end_date = '2022-12-31'
Output_file = 'stock_prices.xlsx'

# Create a pandas DataFrame to store the data
price_data = pdr.get_data_yahoo(
    tickers, start=start_date, end=end_date)['Close']
adj_price_data = pdr.get_data_yahoo(
    tickers, start=start_date, end=end_date)['Adj Close']
volume_data = pdr.get_data_yahoo(
    tickers, start=start_date, end=end_date)['Volume']

In [ ]:
# Convert the datetime index to string in the format "MM/DD/YYYY"
price_data.index = price_data.index.strftime('%m/%d/%Y')
adj_price_data.index = adj_price_data.index.strftime('%m/%d/%Y')
volume_data.index = volume_data.index.strftime('%m/%d/%Y')

In [ ]:

# Create an Excel writer
with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
    # Save close prices to "Price_daily" sheet
    price_data.to_excel(writer, sheet_name='Price_daily')

    # Save adjusted close prices to "Adj_Price_daily" sheet
    adj_price_data.to_excel(writer, sheet_name='Adj_Price_daily')

    # Save volume data to "Volume_daily" sheet
    volume_data.to_excel(writer, sheet_name='Volume_daily')

print("Stock prices saved to stock_prices.xlsx")

In [ ]:
sp500_data = pdr.get_data_yahoo('^GSPC', start=start_date, end=end_date)[
    ["Adj Close", "Close", "High", "Low", "Open", "Volume"]]
sp500_data.index = sp500_data.index.strftime('%m/%d/%Y')

with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
    sp500_data.to_excel(writer, sheet_name='S&P 500')
print("Prices for the S&P 500 index (^GSPC) saved to stock_prices.xlsx")

# Calculating Market Captilization


In [ ]:
from Market_Cap import cal_market_cap, filter_last_date_per_year

constituents_file = "S&P 500 Constituent.xlsx"
cal_market_cap(start_date, end_date, constituents_file, Output_file, tickers)


sheet_name = 'Market_Caps'
filtered_df = filter_last_date_per_year(Output_file, sheet_name)


with pd.ExcelWriter(Output_file, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
    filtered_df.to_excel(writer, sheet_name='Market_Caps')

print("Annual maket capital saved to stock_prices.xlsx")

# Calculating annual liquidity


In [ ]:
from liquidity import cal_liquidity
excel_file = "Stock Data Output.xlsx"  # Replace with your file path
cal_liquidity(Output_file, tickers, constituents_file)

# Calculating returns


In [ ]:
from returns import cal_returns

cal_returns(Output_file, tickers, adj_price_data, start_date, end_date)

# Calculating Risks

In [ ]:
def calculate_returns(prices):
    return prices.pct_change()

# Date range
start_date = "2002-01-01"
end_date = "2022-12-31"
excel_file = "Stock Data Output.xlsx"


def main():
    # Create a new Excel file or open existing file
    with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        for data_type in ["Adj_Price_daily"]:
            # Create an empty DataFrame to store combined data for all companies
            combined_data = pd.DataFrame()

            for ticker in company_names:
                print(f"Downloading {data_type} for {ticker}...")
                data = download_data(ticker, start_date, end_date)

                # Extract relevant column for the current data type
                if data_type == "Adj_Price_daily":
                    current_data = data["Adj Close"]

                # Combine data for all tickers
                combined_data[ticker] = current_data

            # Write combined data to a single sheet
            combined_data.to_excel(writer, sheet_name=data_type)

            # Calculate returns
            returns_daily = calculate_returns(combined_data)

            # Append returns to existing or new sheets
            returns_daily.to_excel(writer, sheet_name="Returns_daily", index=True, header=True)

            # Calculate and save standard deviation of each stock's returns for each year
            risk_annual = returns_daily.groupby(returns_daily.index.year).std()
            risk_annual.to_excel(writer, sheet_name="Risk_annual", index=True, header=True)

    print("Data download, Excel file update, returns calculation, and risk calculation completed.")

if __name__ == "__main__":
    main()

# Calculating Volatility etc.

In [ ]:
# Load the data from Excel file
file_path1 = 'Stock Data Output.xlsx'
returns_sheet_name = 'Returns_annual'

constituents_sheet_name = 'S&P 500 Constituent'

returns_data = pd.read_excel(file_path1, sheet_name=returns_sheet_name, index_col=0)
constituents_data = pd.read_excel("Company_Student_List .xlsx", sheet_name=constituents_sheet_name, index_col=0)

# Filter returns data for the last 5 years (2018:2022)
returns_data_last_5_years = returns_data.loc['2018-01-01':'2022-12-31']

# Create a new DataFrame for summary statistics
summary_stats_df = pd.DataFrame()


# Calculate and add summary statistics for each firm
summary_stats_df['Min'] = returns_data_last_5_years.min()
summary_stats_df['Max'] = returns_data_last_5_years.max()
summary_stats_df['Mean'] = returns_data_last_5_years.mean()
summary_stats_df['Volatility'] = returns_data_last_5_years.std()

# Load market capitalization data
market_cap_data = pd.read_excel(file_path1, sheet_name="Market_Cap", index_col=0)

# Add market capitalization (size) for each firm to the summary_stats_df
summary_stats_df['Size'] = market_cap_data.mean()  # You can use mean() or any other aggregation method



# Add industry information for each firm
summary_stats_df['Industry'] = constituents_data['GICS Sector']

# Add a new sheet "Firm_Summary_Stat" to the existing Excel file
with pd.ExcelWriter(file_path1, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    summary_stats_df.T.to_excel(writer, sheet_name="Firm_Summary_Stat", index=True, header=True)


def calculate_beta(stock_symbol, benchmark_symbol, start_date, end_date):
    # Download historical stock prices
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date, interval='1mo')
    benchmark_data = yf.download(benchmark_symbol, start=start_date, end=end_date, interval='1mo')

    # Extract adjusted closing prices
    stock_adj_close = stock_data['Adj Close']
    benchmark_adj_close = benchmark_data['Adj Close']

    # Calculate monthly returns
    stock_returns = stock_adj_close.pct_change().dropna()
    benchmark_returns = benchmark_adj_close.pct_change().dropna()

    # Calculate covariance and variance
    covariance = np.cov(stock_returns, benchmark_returns)[0, 1]
    variance = np.var(benchmark_returns)

    # Calculate beta
    beta = covariance / variance

    return beta

ticker_list = company_names

# Benchmark symbol (e.g., S&P 500)
benchmark_symbol = "^GSPC"

# Date range
start_date = "2018-01-01"
end_date = "2022-01-01"

# Store beta values calculated from download in a dictionary
downloaded_betas = {}
for stock_symbol in ticker_list:
    beta_value = calculate_beta(stock_symbol, benchmark_symbol, start_date, end_date)
    downloaded_betas[stock_symbol] = beta_value

# Replace 'your_tickers_list' with your list of tickers
given_tickers = company_names


# Replace 'Company_Student_List.xlsx' with the path to your Excel file
excel_file_path = "Company_Student_List .xlsx"

# Read the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Get the required sheet
sheet_name = 'S&P 500 Constituent'
sheet_data = excel_data.parse(sheet_name)

# Get the column 'Market Beta on Nov 2023'
market_beta_column = 'Market Beta on Nov 2023'

# Filter rows based on the given tickers
filtered_rows = sheet_data[sheet_data['ticker'].isin(given_tickers)]

# Display the 'Ticker' and 'Market Beta on Nov 2023' columns for the filtered rows
for index, row in filtered_rows.iterrows():
    ticker = row['ticker']
    excel_beta = row[market_beta_column]
    
    # Check if the ticker has a downloaded beta value
    if ticker in downloaded_betas:
        downloaded_beta = downloaded_betas[ticker]
        
        if (downloaded_beta>excel_beta):
            percentage_difference = ((downloaded_beta - excel_beta) / max(abs(excel_beta), abs(downloaded_beta))) * 100
        elif(excel_beta>downloaded_beta):
            percentage_difference = ((excel_beta - downloaded_beta) / max(abs(excel_beta), abs(downloaded_beta))) * 100


        # Calculate the difference in percentage


        # Update the "Firm_Summary_Stat" sheet with Excel Beta, Downloaded Beta, and Percentage Difference
        with pd.ExcelWriter(file_path1, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
            summary_stats_df.at[ticker, 'Excel Beta'] = excel_beta
            summary_stats_df.at[ticker, 'Downloaded Beta'] = downloaded_beta
            summary_stats_df.at[ticker, 'Percentage Difference'] = percentage_difference
            summary_stats_df.T.to_excel(writer, sheet_name="Firm_Summary_Stat", index=True, header=True)
        
        # Print the results
        #print(f"Ticker: {ticker}, Downloaded Beta: {downloaded_beta}, Excel Beta: {excel_beta}, Difference (%): {percentage_difference}")
    else:
        print(f"Ticker: {ticker}, Downloaded Beta not available")

# Building Personal Portfolio

In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
from pandas.tseries.offsets import MonthEnd
import scipy.optimize as opt
import yfinance as yf
import matplotlib.pyplot as plt

def SharpeRatio(weights, Ret, Rf):
    w = np.append(weights, 1 - sum(weights))
    Ret_portfolio = (w * Ret).sum(axis=1)
    SR_portfolio = -1 * (Ret_portfolio.mean() - Rf) * 12 / Ret_portfolio.std() / np.sqrt(12)
    return SR_portfolio

# Code for the first part
tickers = ['EMN', 'CINF', 'KEYS', 'CDW', 'RVTY', 'KMI', 'COR', 'NXPI', 'INVH', 'EMR', 'BMY', 'NCLH', 'GPC', 'HUM', 'DHI', 'ES', 'CVS', 'GIS', 'AEP', 'AXON', 'CE', 'ZBRA', 'SWKS', 'PEAK', 'SJM']
start_date = '2003-01-01'
end_date = '2022-12-31'
panel_data = yf.download(tickers, start_date, end_date)

# Resample data to monthly frequency and fill missing data with zeros
Returns = panel_data['Adj Close'].resample("1m").ffill().pct_change().fillna(0)

FF_3Factor_All = pdr.get_data_famafrench('F-F_Research_Data_Factors', start_date, end_date)
FF_3Factor = FF_3Factor_All[0] / 100
FF_3Factor.index = FF_3Factor.index.to_timestamp() + MonthEnd(1)
FF_3Factor = FF_3Factor[1:]
Rf = FF_3Factor['RF'].mean()

weights_0 = np.ones(len(tickers) - 1) / len(tickers)
optimal_weights = opt.fmin_bfgs(SharpeRatio, weights_0, args=(Returns[tickers], Rf))

optimal_weights_all = np.append(optimal_weights, 1 - sum(optimal_weights))
optimal_weights_all = pd.Series(optimal_weights_all, index=tickers)
optimal_SharpeRatio = -1 * SharpeRatio(optimal_weights, Returns[tickers], Rf)

# Code for the second part
# Define the ticker symbol for S&P 500 (^GSPC)
ticker_symbol = "^GSPC"

# Download historical data
data_gspc = yf.download(ticker_symbol, start=start_date, end=end_date)

# Resample data to get monthly returns
monthly_returns_gspc = data_gspc['Adj Close'].resample('M').ffill().pct_change()

# Combine Annesha's Fund and GSPC Monthly Return
combined_returns = pd.DataFrame({
    'Date': Returns.index.strftime('%Y-%m'),
    "Annesha's Fund": (Returns[tickers] * optimal_weights_all).sum(axis=1).round(3),
    'GSPC Monthly Return': monthly_returns_gspc.round(3),
    'Year': Returns.index.year
})

# Reset the index before creating cumulative returns
combined_returns.reset_index(drop=True, inplace=True)

# Plot for annual returns as a bar chart
annual_returns = combined_returns.groupby('Year').agg({'Annesha\'s Fund': 'sum', 'GSPC Monthly Return': 'sum'})

plt.figure(figsize=(10, 6))

# Bar chart for annual returns
bar_width = 0.35
plt.bar(annual_returns.index - bar_width/2, annual_returns["Annesha's Fund"], bar_width, label="Annesha's Fund")
plt.bar(annual_returns.index + bar_width/2, annual_returns['GSPC Monthly Return'], bar_width, label='GSPC')

plt.title('Annual Returns')
plt.xlabel('Year')
plt.ylabel('Returns')
plt.legend()
plt.grid(True)

# Save the bar chart as an image
bar_chart_image_path = 'bar_chart.png'
plt.savefig(bar_chart_image_path)
plt.close()

# Create a cumulative returns DataFrame
cumulative_returns = combined_returns.copy()
cumulative_returns['Cumulative Annesha\'s Fund'] = cumulative_returns["Annesha's Fund"].cumsum()
cumulative_returns['Cumulative GSPC'] = cumulative_returns['GSPC Monthly Return'].cumsum()

# Plot for cumulative returns as a step plot
plt.figure(figsize=(10, 6))

# Line chart for cumulative returns
plt.step(cumulative_returns.index, cumulative_returns["Cumulative Annesha's Fund"], label="Cumulative Annesha's Fund", where='post')
plt.step(cumulative_returns.index, cumulative_returns['Cumulative GSPC'], label='Cumulative GSPC', where='post')

plt.title('Cumulative Returns')
plt.xlabel('Month')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid(True)

# Save the cumulative returns chart as an image
cumulative_chart_image_path = 'cumulative_chart.png'
plt.savefig(cumulative_chart_image_path)
plt.close()

# Save the output to an existing Excel file with the plots
output_file_path = 'Stock Data Output.xlsx'
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a',if_sheet_exists="replace") as writer:
    # Save the combined returns to the sheet "PortfolioReturn_monthly"
    combined_returns.to_excel(writer, index=False, sheet_name='PortfolioReturn_monthly')

    # Save the bar chart image to the same worksheet
    worksheet_bar_chart = writer.sheets['PortfolioReturn_monthly']
    image_width = 400
    image_height = 300
    #worksheet_bar_chart.insert_image('F2', bar_chart_image_path, {'x_scale': image_width / 100, 'y_scale': image_height / 100})

    # Save the cumulative returns chart image to the same worksheet
    worksheet_cumulative_chart = writer.sheets['PortfolioReturn_monthly']
    #worksheet_cumulative_chart.insert_image('M2', cumulative_chart_image_path, {'x_scale': image_width / 100, 'y_scale': image_height / 100})

print(f'Output saved to {output_file_path}')


# Calculating portfolio performance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

# Calculate additional metrics
portfolio_returns = combined_returns["Annesha's Fund"]
benchmark_returns = combined_returns['GSPC Monthly Return']

portfolio_returns = portfolio_returns.fillna(0)
benchmark_returns = benchmark_returns.fillna(0)

# Check if there are NaN values after dropping them
if portfolio_returns.isnull().any() or benchmark_returns.isnull().any():
    raise ValueError("NaN values are present in portfolio or benchmark returns after dropping.")

# Calculate mean, std, min, max
mean_return = portfolio_returns.mean()
std_return = portfolio_returns.std()
min_return = portfolio_returns.min()
max_return = portfolio_returns.max()

# Calculate Alpha, Beta, R-squared
X = benchmark_returns.values.reshape(-1, 1)
y = portfolio_returns.values

# Check for NaN values in X and y
if np.isnan(X).any() or np.isnan(y).any():
    raise ValueError("NaN values are present in X or y after dropping.")

# Check if there are NaN values after dropping them
if portfolio_returns.isnull().any() or benchmark_returns.isnull().any():
    raise ValueError("NaN values are present in portfolio or benchmark returns after dropping.")

model = LinearRegression().fit(X, y)
alpha = model.intercept_
beta = model.coef_[0]
r_squared = model.score(X, y)

# Check if there are NaN values in the calculated metrics
if np.isnan(alpha) or np.isnan(beta) or np.isnan(r_squared):
    raise ValueError("NaN values are present in calculated metrics.")

# Calculate Sharpe ratio
sharpe_ratio = (mean_return - Rf) / std_return

# Calculate Treynor ratio
treynor_ratio = (mean_return - Rf) / beta

# Print the metrics
print(f"Mean Return: {mean_return:.4f}")
print(f"Standard Deviation (in %): {std_return * 100:.4f}")
print(f"Minimum Return (in %): {min_return * 100:.4f}")
print(f"Maximum Return (in %): {max_return * 100:.4f}")
print(f"Alpha: {alpha:.4f}")
print(f"Beta: {beta:.4f}")
print(f"R-squared: {r_squared:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Treynor Ratio: {treynor_ratio:.4f}")

# Plot a histogram of portfolio returns with lines for Annesha Fund and GSPC
plt.figure(figsize=(10, 6))

# Plot histogram
hist, bins, _ = plt.hist(portfolio_returns, bins=20, color='white', edgecolor='maroon', alpha=0.7,histtype='step', label="Annesha's Fund", density=True)
plt.hist(benchmark_returns, bins=bins, color='white', edgecolor='blue', alpha=0.7, label='GSPC', density=True,histtype='step')

plt.title('Histogram of Portfolio Returns')
plt.xlabel('Monthly Returns')
plt.ylabel('Frequency')
plt.legend()
plt.grid(False)
plt.show()


# Calculating Portfolio Returns

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

# Calculate additional metrics
portfolio_returns = combined_returns["Annesha's Fund"]
benchmark_returns = combined_returns['GSPC Monthly Return']

portfolio_returns = portfolio_returns.fillna(0)
benchmark_returns = benchmark_returns.fillna(0)

# Check if there are NaN values after dropping them
if portfolio_returns.isnull().any() or benchmark_returns.isnull().any():
    raise ValueError("NaN values are present in portfolio or benchmark returns after dropping.")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Calculate additional metrics
portfolio_returns = combined_returns["Annesha's Fund"]
benchmark_returns = combined_returns['GSPC Monthly Return']

portfolio_returns = portfolio_returns.fillna(0)
benchmark_returns = benchmark_returns.fillna(0)

# Check if there are NaN values after dropping them
if portfolio_returns.isnull().any() or benchmark_returns.isnull().any():
    raise ValueError("NaN values are present in portfolio or benchmark returns after dropping.")

# Calculate mean, std, min, max in annual percentage rates
mean_return_annual = (1 + portfolio_returns).prod() ** (12 / len(portfolio_returns.index)) - 1
std_return_annual = portfolio_returns.std() * np.sqrt(12)
min_return_annual = portfolio_returns.min() * 12
max_return_annual = portfolio_returns.max() * 12

# Convert to percentage for display
mean_return_annual_percent = mean_return_annual * 100
std_return_annual_percent = std_return_annual * 100
min_return_annual_percent = min_return_annual * 100
max_return_annual_percent = max_return_annual * 100

# Calculate Alpha, Beta, R-squared for Annesha Fund
X_portfolio = benchmark_returns.values.reshape(-1, 1)
y_portfolio = portfolio_returns.values

# Check for NaN values in X and y
if np.isnan(X_portfolio).any() or np.isnan(y_portfolio).any():
    raise ValueError("NaN values are present in X or y for Annesha Fund after dropping.")

# Check if there are NaN values after dropping them
if portfolio_returns.isnull().any() or benchmark_returns.isnull().any():
    raise ValueError("NaN values are present in Annesha Fund or benchmark returns after dropping.")

model_portfolio = LinearRegression().fit(X_portfolio, y_portfolio)
alpha_portfolio = model_portfolio.intercept_
beta_portfolio = model_portfolio.coef_[0]
r_squared_portfolio = model_portfolio.score(X_portfolio, y_portfolio)

# Calculate Sharpe ratio and Treynor ratio for Annesha Fund
sharpe_ratio_portfolio = (mean_return_annual - Rf) / std_return_annual
treynor_ratio_portfolio = (mean_return_annual - Rf) / beta_portfolio

# Calculate Alpha, Beta, R-squared for GSPC
X_gspc = benchmark_returns.values.reshape(-1, 1)
y_gspc = benchmark_returns.values

# Check for NaN values in X and y
if np.isnan(X_gspc).any() or np.isnan(y_gspc).any():
    raise ValueError("NaN values are present in X or y for GSPC after dropping.")

model_gspc = LinearRegression().fit(X_gspc, y_gspc)
alpha_gspc = model_gspc.intercept_
beta_gspc = model_gspc.coef_[0]
r_squared_gspc = model_gspc.score(X_gspc, y_gspc)

# Calculate Sharpe ratio and Treynor ratio for GSPC
sharpe_ratio_gspc = (benchmark_returns.mean() * 12 - Rf) / (benchmark_returns.std() * np.sqrt(12))
treynor_ratio_gspc = (benchmark_returns.mean() * 12 - Rf) / beta_gspc

# Create a DataFrame to store the results
summary_data = pd.DataFrame({
    'Variable': ['Average', 'Standard Deviation', 'Minimum', 'Maximum', 'Alpha', 'Beta', 'R-squared', 'Sharpe Ratio', 'Treynor Ratio'],
    "Annesha Fund": [f'{mean_return_annual_percent:.2f}%', f'{std_return_annual_percent:.2f}%', f'{min_return_annual_percent:.2f}%', f'{max_return_annual_percent:.2f}%', alpha_portfolio, beta_portfolio, r_squared_portfolio, sharpe_ratio_portfolio, treynor_ratio_portfolio],
    'GSPC': [f'{benchmark_returns.mean() * 12 * 100:.2f}%', f'{benchmark_returns.std() * np.sqrt(12) * 100:.2f}%', f'{benchmark_returns.min() * 12 * 100:.2f}%', f'{benchmark_returns.max() * 12 * 100:.2f}%', alpha_gspc, beta_gspc, r_squared_gspc, sharpe_ratio_gspc, treynor_ratio_gspc]
})

# Save the summary data to Excel
excel_file_path = 'Stock Data Output.xlsx'
with pd.ExcelWriter(excel_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    summary_data.to_excel(writer, sheet_name='Fund_summary', index=False)

# Plot a histogram of portfolio returns with lines for Annesha Fund and GSPC
plt.figure(figsize=(10, 6))

# Plot histogram
hist, bins, _ = plt.hist(portfolio_returns, bins=20, color='white', edgecolor='maroon', alpha=0.7, histtype='step', label="Annesha Fund", density=True)
plt.hist(benchmark_returns, bins=bins, color='white', edgecolor='blue', alpha=0.7, label='GSPC', density=True, histtype='step')

plt.title('Histogram of Portfolio Returns')
plt.xlabel('Monthly Returns')
plt.ylabel('Frequency')
plt.legend()
plt.grid(False)
plt.savefig('Histogram_Portfolio_Returns.png')
plt.show()



# Calculating investment summary

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import scipy.optimize as opt
from pandas.tseries.offsets import MonthEnd

# Function to calculate Sharpe Ratio
def SharpeRatio(weights, Ret, Rf):
    w = np.append(weights, 1 - sum(weights))
    Ret_portfolio = (w * Ret).sum(axis=1)
    SR_portfolio = -1 * (Ret_portfolio.mean() - Rf) * 12 / Ret_portfolio.std() / np.sqrt(12)
    return SR_portfolio

# Define the tickers and other parameters
tickers = ['EMN', 'CINF', 'KEYS', 'CDW', 'RVTY', 'KMI', 'COR', 'NXPI', 'INVH', 'EMR', 'BMY', 'NCLH', 'GPC', 'HUM', 'DHI', 'ES', 'CVS', 'GIS', 'AEP', 'AXON', 'CE', 'ZBRA', 'SWKS', 'PEAK', 'SJM']
start_date = '2003-01-01'
end_date = '2023-11-30'  # Updated end date

# Download data
panel_data = yf.download(tickers, start_date, end_date)

# Resample data to monthly frequency and fill missing data with zeros
Returns = panel_data['Adj Close'].resample("1m").ffill().pct_change().fillna(0)

# Set initial weights
weights_0 = np.ones(len(tickers) - 1) / len(tickers)

# Optimize weights to maximize Sharpe Ratio
optimal_weights = opt.fmin_bfgs(SharpeRatio, weights_0, args=(Returns[tickers], Rf))

# Calculate final weights with cash
optimal_weights_all = np.append(optimal_weights, 1 - sum(optimal_weights))
optimal_weights_all = pd.Series(optimal_weights_all, index=tickers)

# Calculate investments in each firm at the end of the sample
investments = panel_data['Adj Close'].iloc[-1] * optimal_weights_all

# Ensure that investments are non-negative
investments[investments < 0] = 0

# Calculate percentage investments
percentage_investments = (investments / investments.sum()) * 100

# Preserve the original order of tickers
percentage_investments = percentage_investments.loc[tickers]

# Display investments
print("Investments at the end of the sample:")
print(percentage_investments.round(2))

# Save the summary data to Excel
excel_file_path = 'Company_Student_List .xlsx'

# Create a DataFrame to store the results
summary_data = pd.DataFrame({'Symbols': tickers, end_date: percentage_investments.round(2)})
# Append the summary_data to the 'fund_summary2' sheet in the Excel file
with pd.ExcelWriter("Stock Data Output.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    summary_data.to_excel(writer, sheet_name='fund_summary2', index=False)

# Load industry data from Excel
industry_data = pd.read_excel(excel_file_path, sheet_name='S&P 500 Constituent', usecols=['ticker', 'GICS Sector'])

# Merge industry data with summary_data
summary_data = pd.merge(summary_data, industry_data, left_on='Symbols', right_on='ticker', how='left')

# Group by industry and sum investments
industry_investments = summary_data.groupby('GICS Sector')[end_date].sum()

# Plot a pie chart with leader lines for industries
fig, ax = plt.subplots(figsize=(12, 8))
wedges, texts, autotexts = ax.pie(industry_investments, autopct='%1.1f%%', textprops=dict(color="w"))

# Add leader lines
for i, text in enumerate(texts):
    angle = (wedges[i].theta2 - wedges[i].theta1) / 2. + wedges[i].theta1
    x = np.cos(np.radians(angle))
    y = np.sin(np.radians(angle))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = f"angle,angleA=0,angleB={angle}"
    ax.annotate(f'{industry_investments.index[i]}', xy=(x, y), xytext=(1.35 * np.sign(x), 1.4 * y),
                horizontalalignment=horizontalalignment, arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color='black'))

plt.title('Industry Composition at the End of the Sample')
plt.savefig('industry_pie_chart.png')
plt.show()


# Making excel file more readalbe.

###### (This is the last step)


#### Wraping Text
